#Introduction

This notebook has all the code you need to create your own chatbot with custom knowledge base using GPT-3.

Follow the instructions for each steps and then run the code sample. In order to run the code, you need to press "play" button near each code sample.

# Install the dependicies
Run the code below to install the depencies we need for our functions

In [2]:
!pip install llama-index==0.5.6
!pip install langchain==0.0.148

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.9 MB/s eta 0:00:00
  Created wheel for llama-index: filename=llama_index-0.5.6-py3-none-any.whl size=248134 sha256=96272d0b399764b4db73e0e77aca41f0b7d8c859ca0efbad7756093ebb4b7770
  Stored in directory: /root/.cache/pip/wheels/48/9f/0a/a5d7181a1819086f8288d1becdad81de07fc874b55075dde19
Successfully built llama-index
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.7/636.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Define the functions
The following code defines the functions we need to construct the index and query it

In [3]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))

    documents = SimpleDirectoryReader(directory_path).load_data()

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True:
        query = input("What do you want to ask? ")
        response = index.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))

In [12]:
from IPython.display import display, Markdown


def ask_ai(prompt="Find the best route based on trained verizon links. I have trained the data of for each url for verizon website, all related words or words present in the url. Now according to the user text, you should find the url that best meets the user text by checking the exact or nearest words given by the user and regex of url by finding related words or phrases in each url. act as a router and route the users to correct route based on text given by users. do not generate more than one word answer to user. just give the most suitable route. if no related route is found, give output as www.verizon.com"):
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True:

        query = input(" what can I help you with?")
        response = index.query(prompt + query)
        display(Markdown(f"Response: <b>{response.response}</b>"))

# Set OpenAI API Key
You need an OPENAI API key to be able to run this code.

If you don't have one yet, get it by [signing up](https://platform.openai.com/overview). Then click your account icon on the top right of the screen and select "View API Keys". Create an API key.

Then run the code below and paste your API key into the text input.

In [4]:
# os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")
os.environ["OPENAI_API_KEY"]="";

#Construct an index
Now we are ready to construct the index. This will take every file in the folder 'data', split it into chunks, and embed it with OpenAI's embeddings API.

**Notice:** running this code will cost you credits on your OpenAPI account ($0.02 for every 1,000 tokens). If you've just set up your account, the free credits that you have should be more than enough for this experiment.

In [5]:
construct_index("data")

In [6]:
ask_ai()

What do you want to ask? according to the data feeded, find the most relevant link for verizon featured smart phones


Response: <b>

https://www.verizon.com/smartphones/</b>

What do you want to ask? how to refer a friend link


Response: <b>
The link to refer a friend is not provided in the context information.</b>

What do you want to ask? refer


...

What do you want to ask? deals


Response: <b>

Verizon offers deals on a variety of products and services, including smartphones, tablets, smartwatches, hotspots, and more. Customers can find deals on the Verizon website, such as free trials, discounts on select products, and trade-in offers. Additionally, Verizon provides consumer guides, such as the Cell Phone Buyers Guide and the Pick the Perfect Otterbox Case for You guide, to help customers find the best deals for their needs.</b>

What do you want to ask? deals link


Response: <b>

https://www.verizon.com/plans/55plus/
https://www.verizon.com/featured/unlocked-phones/
https://www.verizon.com/solutions-and-services/apple-trade-in/
https://www.verizon.com/solutions-and-services/apple-iphone-trade-in/
https://www.verizon.com/solutions-and-services/apple-watch-trade-in/
https://www.verizon.com/solutions-and-services/apple-ipad-trade-in/
https://www.verizon.com/solutions-and-services/smartphones-trade-in/
https://www.verizon.com/shop/consumer-guides/the-cell-phone-buyers-guide
https://www.verizon.com/articles/Accessories/pick-the-perfect-otterbox-case-for-you/</b>

KeyboardInterrupt: ignored

In [11]:
ask_ai()


Find the best route. I have trained the data of for each url, all related words or words present in the url. Now according to the user text, you should find the url that best meets the user text by checking the exact or nearest words given by the user and finding them in each url. act as a router and route the users to correct route based on text given by users refer a friend


Response: <b>

https://www.verizon.com/refer-a-friend/</b>

Find the best route. I have trained the data of for each url, all related words or words present in the url. Now according to the user text, you should find the url that best meets the user text by checking the exact or nearest words given by the user and finding them in each url. act as a router and route the users to correct route based on text given by users fastest internet


Response: <b>

The fastest internet offered by Verizon is Fios Fastest Internet, which is available with Fios TV Voice Remote, Fios Remote Control 2 Device, and Fios Voice Backup Battery.</b>

Find the best route. I have trained the data of for each url, all related words or words present in the url. Now according to the user text, you should find the url that best meets the user text by checking the exact or nearest words given by the user and finding them in each url. act as a router and route the users to correct route based on text given by users fastest internet link


...

Find the best route. I have trained the data of for each url, all related words or words present in the url. Now according to the user text, you should find the url that best meets the user text by checking the exact or nearest words given by the user and finding them in each url. act as a router and route the users to correct route based on text given by users 5g


Response: <b>

5G is a fifth-generation cellular network technology that provides faster speeds and more reliable connections than previous generations of cellular networks. It is available on many smartphones, including the Samsung Galaxy S23 Plus, Samsung Galaxy A14 5G, Samsung Galaxy A23 5G UW, and Google Pixel 7. To ensure your device is 5G-ready, check out Verizon's Cell Phone Buyer's Guide and pick the perfect Otterbox case for your device.</b>

Find the best route. I have trained the data of for each url, all related words or words present in the url. Now according to the user text, you should find the url that best meets the user text by checking the exact or nearest words given by the user and finding them in each url. act as a router and route the users to correct route based on text given by users 5g link


Response: <b>

https://www.verizon.com/support/software-updates-5g-ultrawideband/?adobe_mc=MCMID%3D32705666790790499832660074436097671344%7CMCORGID%3D843F02BE53271A1A0A490D4C%2540AdobeOrg%7CTS%3D1604013094&amp;mboxSession=c2b27266cccc461790c393e36c3173a0</b>

KeyboardInterrupt: ignored